In [1]:
#YANK BP
import textwrap, sys, os, glob, shutil, openmm
#import nglview
#import openmmtools as mmtools
#import yank.utils
import numpy as np
import MDAnalysis as mda
#OPENFF
from openff.units import Quantity, unit
from openmm import unit as openmm_unit
from pdbfixer import PDBFixer
from openff.toolkit import ForceField, Molecule, Topology
from openbabel import openbabel

In [2]:
job_inputs = {'working_dir_name': 'bromodomains_testing',
              'ligand_resname': 'resname JQ1',
              'crystal_pdb_fn': '3mxf.pdb'}

## Steps to Prepare

In [3]:
class Simulation_Preparer():
    def __init__(self, job_inputs):
        #Declare Filenames
        self.job_inputs = job_inputs
        working_dir_name = job_inputs['working_dir_name']
        self.ligand_resname = job_inputs['ligand_resname']
        self.crystal_pdb_fn = job_inputs['crystal_pdb_fn']

        self.abs_work_dir = os.path.join(os.getcwd(), working_dir_name)
        if not os.path.isdir(self.abs_work_dir):
            os.mkdir(self.abs_work_dir)

    
    def seperate_crys_using_MDA(self,
                                crys_pdb_fn: str,
                                ligand_string: str,
                                receptor_string: str = 'protein',
                                ligand_pdb_fn: str = 'ligand_crys.pdb',
                                receptor_pdb_fn: str = 'receptor_crys.pdb'):
        """
        Loads the provided crystal structure, and seperates the atoms of resname LIGAND STRING into their own file
        Arguments:
            crys_pdb_fn: crystal structure to be seperated
            ligand_string: MDAnalysis selection string for ligand
            receptor_string: MDAnalysis selection string for receptor (default 'protein')
            ligand_pdb_fn: Write path for MDAnalysis ligand (default 'ligand.pdb')
            receptor_pdb_fn: Write path for MDAnalysis receptor (default 'receptor.pdb')
        Returns:
            receptor_pdb_fn: filename of the pdb file containing the receptor only
            ligand_pdb_fn: filename of the pdb file containing the atoms from the LIGAND_STRING selection
        """
        u = mda.Universe(crys_pdb_fn)
        all_atoms = u.select_atoms('all')
        receptor = all_atoms.select_atoms(receptor_string)
        ligand = all_atoms.select_atoms(ligand_string)
        receptor.write(os.path.join(self.abs_work_dir, receptor_pdb_fn))
        ligand.write(os.path.join(self.abs_work_dir, ligand_pdb_fn))
        return os.path.join(self.abs_work_dir, receptor_pdb_fn), os.path.join(self.abs_work_dir, ligand_pdb_fn)

    def obabel_conversion(self,
                          mol_fn: str,
                          formats: list,
                          out_fn: str = 'AUTO',
                          add_Hs: bool = True,
                          rewrite_with_Hs: bool = True):
        """
        Convert a file to another format using openbabel.  Neither add_Hs, nore rewrite_with_Hs should be True if the input has hydrogens.
        Arguments:
            mol_fn: in_file_name : (THis should be infile.xxx where xxx = formats[0])
            formats: 2-list of babel formats (in, out)
            out_fn: Filename for the ligand with Hydrogens (if AUTO, it will be MOL_FN with the extension swapped to format[-1])
            add_Hs: Bool: Whether adding hydrogens to the input file should be done (default True)
            rewrite_with_Hs: Bool: Additionally rewrites the protonated ligand in the original file format (default True)
        Returns:
            str: The path of the converted file
        Example:
            obabel_conversion('ligand.pdb', ['pdb', 'sdf']) will attempt to protonate (by default) and convert a pdb file to sdf (named ligand.sdf)
        """
        #Assertion checks
        assert len(formats) == 2
        #Obabel conversion block
        obConversion = openbabel.OBConversion()
        obConversion.SetInAndOutFormats(*formats)
        mol = openbabel.OBMol()
        #Find INput
        if os.path.isfile(mol_fn):
            obConversion.ReadFile(mol, mol_fn)
        elif os.path.isfile(os.path.join(self.abs_work_dir, mol_fn)):
            obConversion.ReadFile(mol, os.path.join(self.abs_work_dir, mol_fn))
        else:
            raise FileNotFoundError('mol_fn was not found')
        #Add Hydrogens
        if add_Hs:
            mol.AddHydrogens()
        print(mol.NumAtoms(), 'Atoms', mol.NumBonds(), 'Bonds', mol.NumResidues(), 'Residues')
        #Output file name parsing
        if out_fn == 'AUTO':
            out_fn = os.path.splitext(mol_fn)[0] + '.' + formats[-1]
        #Actually writeout the protonated file in the second format
        obConversion.WriteFile(mol, out_fn)
        
        #Rewrite original format with Hydrogens (if necessary)
        if rewrite_with_Hs:
            #recursively use this function to convert from format 0 to format 0 again
            org_form_new_fn = os.path.splitext(mol_fn)[0] + '_H.' + formats[0]
            org_form_wHs_fn = self.obabel_conversion(mol_fn, [formats[0], formats[0]], out_fn=org_form_new_fn, add_Hs=True, rewrite_with_Hs=False)[0]
            return (out_fn, org_form_wHs_fn)
        else:
            return (out_fn, None)

    def protonate_with_pdb2pqr(self,
                               protein_fn: str,
                               protein_H_fn: str = None,
                               at_pH=7):
        """Protonates the given structure using pdb2pqr30
        Parameters:
            protein_fn: structure to be protonated
        protein_H_fn: filepath for protonated receptor (as pdb)
                        pqr output of pdb2pqr is inferred as protein_H_fn with a pqr extension instead of pdb
        at_pH: pH for protonation (default 7)
        """
        if protein_H_fn is None:
            protein_H_fn = os.path.splitext(protein_fn)[0] + '_H' + os.path.splitext(protein_fn)[-1]
        protein_pqr_fn = os.path.splitext(protein_H_fn)[0] + '.pqr'
        my_cmd = f'pdb2pqr30 --ff AMBER --nodebump --keep-chain --ffout AMBER --pdb-output {protein_H_fn} --with-ph {at_pH} {protein_fn} {protein_pqr_fn}'
        print('Protanting using command line')
        print(f'Running {my_cmd}')
        exit_status = os.system(my_cmd)
        print(f'Done with exit status {exit_status}')
        return protein_H_fn, protein_pqr_fn

    def run_PDBFixer(self,
                     in_pdbfile: str,
                     mode: str = "MEM",
                     out_file_fn: str = None,
                     padding = 1.5,
                     ionicStrength = 0.15):
        """
        Generates a solvated and membrane-added system (depending on MODE)
        MODE = 'MEM' for membrane and solvent
        MODE = 'SOL' for solvent only
        Parameters:
            in_pdbfile: the structure to be solvated
            mode: string: must be in ['MEM', 'SOL']
            solvated_file_fn: Filename to save solvated system; default is to add '_solvated' between the body and extension of the input file name
            padding: float or int: minimum nanometer distance between the boundary and any atoms in the input.  Default 1.5 nm = 15 A
            ionicStrength: float (not int) : molar strength of ionic solution. Default 0.15 M = 150 mmol
        Returns:
            solvated_file_fn: the filename of the solvated output
        """
        assert mode in ['MEM', 'SOL']
        fixer = PDBFixer(in_pdbfile)

        if mode == 'MEM':
            fixer.addMembrane('POPE', padding=padding * openmm_unit.nanometer, ionicStrength=ionicStrength * openmm_unit.molar)
        elif mode == 'SOL':
            fixer.addSolvent(padding=padding * openmm_unit.nanometer, ionicStrength=ionicStrength * openmm_unit.molar)
        
        # ADD PDBFixer hydrogens and parsing crystal structures (Hydrogens with pdb2pqr30 at the moment)
        if out_file_fn is None:
            out_file_fn = os.path.splitext(in_pdbfile)[0] + f'_{mode}' + os.path.splitext(in_pdbfile)[-1]
        
        with open(out_file_fn, "w") as f:
            openmm.app.PDBFile.writeFile(fixer.topology, fixer.positions, f)
        return out_file_fn

    def insert_molecule_and_remove_clashes(self,
                                           topology: Topology,
                                           insert: Molecule,
                                           radius: Quantity = 1.5 * unit.angstrom,
                                           keep: list[Molecule] = []) -> Topology:
        """
        Add a molecule to a copy of the topology, removing any clashing molecules.
    
        The molecule will be added to the end of the topology. A new topology is
        returned; the input topology will not be altered. All molecules that
        clash will be removed, and each removed molecule will be printed to stdout.
        Users are responsible for ensuring that no important molecules have been
        removed; the clash radius may be modified accordingly.
    
        Parameters
        ==========
        top
            The topology to insert a molecule into
        insert
            The molecule to insert
        radius
            Any atom within this distance of any atom in the insert is considered
            clashing.
        keep
            Keep copies of these molecules, even if they're clashing
        """
        # We'll collect the molecules for the output topology into a list
        new_top_mols = []
        # A molecule's positions in a topology are stored as its zeroth conformer
        insert_coordinates = insert.conformers[0][:, None, :]
        for molecule in topology.molecules:
            if any(keep_mol.is_isomorphic_with(molecule) for keep_mol in keep):
                new_top_mols.append(molecule)
                continue
            molecule_coordinates = molecule.conformers[0][None, :, :]
            diff_matrix = molecule_coordinates - insert_coordinates
    
            # np.linalg.norm doesn't work on Pint quantities 😢
            working_unit = unit.nanometer
            distance_matrix = (
                np.linalg.norm(diff_matrix.m_as(working_unit), axis=-1) * working_unit
            )
    
            if distance_matrix.min() > radius:
                # This molecule is not clashing, so add it to the topology
                new_top_mols.append(molecule)
            else:
                print(f"Removed {molecule.to_smiles()} molecule")
    
        # Insert the ligand at the end
        new_top_mols.append(insert)
    
        # This pattern of assembling a topology from a list of molecules
        # ends up being much more efficient than adding each molecule
        # to a new topology one at a time
        new_top = Topology.from_molecules(new_top_mols)
    
        # Don't forget the box vectors!
        new_top.box_vectors = topology.box_vectors
        return new_top
        
    def generate_topologies(self,
                            structure_file: str,
                            ligand_to_insert: Molecule = None,
                            json_save_fn: str = None):
        """
        Convert the final structure files into OpenFF topologies
        Parameters:
            structure_file: the structure file to generate topology for
            ligand_to_insert: If an openff toolkit Molecule type object is given, it will be inserted into the receptor.
            save_as_json: If a filename is provided, a json file of the generated topology will be saved
        Returns:
            top: The generated topology
        """
        #Create the topology of the complex phase
        top = Topology.from_pdb(structure_file)
        #Insert the ligand into this phase and remove clashes
        if ligand_to_insert is not None:
            top = self.insert_molecule_and_remove_clashes(top, ligand_to_insert)
        if json_save_fn is not None:
            with open(json_save_fn, "w") as f:
                print(top.to_json(), file=f)
        return top

    def top2interchange(self, top: Topology, xmls: list):
        """
        Convert an OpenFF Topology into and OpenFF Interchange (Can take a long time!)
        This is the actual step where MD parameters are applied
        Parameters:
            top: the topology to be converted
        Returns:
            interchange: The interchange object which was created
        """
        sage_ff14sb = ForceField(*xmls)
        return sage_ff14sb.create_interchange(top)

    def interchange2OpenmmSim(self,
                              interchange,
                             temp: openmm_unit):
        """
        Construct an openn simulation object from an openff interachange
        
        """
        # Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
        integrator = openmm.LangevinIntegrator(300 * openmm_unit.kelvin, 1 / openmm_unit.picosecond, 0.001 * openmm_unit.picoseconds)
        # Under the hood, this creates *OpenMM* `System` and `Topology` objects, then combines them together
        simulation = interchange.to_openmm_simulation(integrator=integrator)
        # Add a reporter to record the structure every 10 steps
        dcd_reporter = openmm.app.DCDReporter(f"trajectory.dcd", 1000)
        simulation.reporters.append(dcd_reporter)
        simulation.reporters.append(openmm.app.StateDataReporter(sys.stdout, 1000, step=True, potentialEnergy=True, temperature=True))
        #Evaluate and Report pre-minimized energy
        describe_state(simulation.context.getState(getEnergy=True, getForces=True), "Original state")
        #Minimize the structure
        simulation.minimizeEnergy()
        #Evaluate and Report post-minimized energy
        describe_state(simulation.context.getState(getEnergy=True, getForces=True), "Minimized state")
        simulation.context.setVelocitiesToTemperature(300 * openmm_unit.kelvin)
        return simulation

Invoke the Class

In [4]:
self = Simulation_Preparer(job_inputs)

Using the provided crystal structure, seperate into protein and ligand components.  THis is based on selection strings, by default the "receptor" is anything under the selection string "protein" and the "ligand" is the user provided selection string for "ligand_resname"

In [5]:
receptor_path, ligand_path = self.seperate_crys_using_MDA(self.crystal_pdb_fn, self.ligand_resname)
print(receptor_path, os.path.isfile(receptor_path))
print(ligand_path, os.path.isfile(ligand_path))

/media/volume/sdc/githubs/MinhSimPrep/bromodomains_testing/receptor_crys.pdb True
/media/volume/sdc/githubs/MinhSimPrep/bromodomains_testing/ligand_crys.pdb True


/home/exouser/miniconda3/envs/openff/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


Utilize openbabel to convert the ligand file to sdf (the prefered input format for OpenFF), while also adding hydrogens

In [6]:
ligand_sdf_path, ligand_protonated = self.obabel_conversion(ligand_path, ['pdb', 'sdf'], out_fn='AUTO', add_Hs=True, rewrite_with_Hs=True)
print(ligand_protonated, os.path.isfile(ligand_protonated))
print(ligand_sdf_path, os.path.isfile(ligand_sdf_path))

56 Atoms 59 Bonds 1 Residues
56 Atoms 59 Bonds 1 Residues
/media/volume/sdc/githubs/MinhSimPrep/bromodomains_testing/ligand_crys_H.pdb True
/media/volume/sdc/githubs/MinhSimPrep/bromodomains_testing/ligand_crys.sdf True


Define the Ligand as an OpenFF molecule

In [7]:
self.ligand = Molecule.from_file(ligand_sdf_path)
print(type(self.ligand), self.ligand.to_smiles(explicit_hydrogens=False))

<class 'openff.toolkit.topology.molecule.Molecule'> Cc1sc2c(c1C)C(c1ccc(Cl)cc1)=N[C@@H](CC(=O)OC(C)(C)C)c1nnc(C)n1-2


Protonate the protiein using pdb2pqr30 on the command line

In [8]:
protein_protonated, protein_pqr = self.protonate_with_pdb2pqr(receptor_path, at_pH=7)
print(protein_protonated, os.path.isfile(protein_protonated))
print(protein_pqr, os.path.isfile(protein_pqr))

Protanting using command line
Running pdb2pqr30 --ff AMBER --nodebump --keep-chain --ffout AMBER --pdb-output /media/volume/sdc/githubs/MinhSimPrep/bromodomains_testing/receptor_crys_H.pdb --with-ph 7 /media/volume/sdc/githubs/MinhSimPrep/bromodomains_testing/receptor_crys.pdb /media/volume/sdc/githubs/MinhSimPrep/bromodomains_testing/receptor_crys_H.pqr


INFO:PDB2PQR v3.6.2: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: /media/volume/sdc/githubs/MinhSimPrep/bromodomains_testing/receptor_crys.pdb
ERROR:Error parsing line: invalid literal for int() with base 10: ''
ERROR:<REMARK     2>
ERROR:Truncating remaining errors for record type:REMARK

ERROR:['REMARK']
INFO:Setting up molecule.
INFO:Created biomolecule object with 127 residues and 1059 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 3 missing atoms in biomolecule.
I

Done with exit status 0
/media/volume/sdc/githubs/MinhSimPrep/bromodomains_testing/receptor_crys_H.pdb True
/media/volume/sdc/githubs/MinhSimPrep/bromodomains_testing/receptor_crys_H.pqr True


INFO:Applying force field to biomolecule states.
INFO:Applying custom naming scheme (amber).
INFO:Regenerating headers.
INFO:Regenerating PDB lines.


Solvate the receptor (without the ligand still)

In [9]:
self.protein_solvated = self.run_PDBFixer(protein_protonated, mode='SOL', padding=1.5, ionicStrength=0.15)
print(self.protein_solvated, os.path.isfile(self.protein_solvated))

/media/volume/sdc/githubs/MinhSimPrep/bromodomains_testing/receptor_crys_H_SOL.pdb True


Solvate the ligand in it's own small box (should it be needed as a second phase of FE calculation)

In [10]:
ligand_solvated = self.run_PDBFixer(ligand_protonated, mode='SOL', padding=1.5, ionicStrength=0.15)
print(ligand_solvated, os.path.isfile(ligand_solvated))

/home/exouser/miniconda3/envs/openff/lib/python3.12/site-packages/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (HETATM   33  H   JQ1 A   1      29.489  18.365   2.512  1.00  0.00           H  , HETATM   32  H   JQ1 A   1      30.644  19.618   2.124  1.00  0.00           H  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/home/exouser/miniconda3/envs/openff/lib/python3.12/site-packages/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (HETATM   34  H   JQ1 A   1      29.120  19.540   1.272  1.00  0.00           H  , HETATM   33  H   JQ1 A   1      29.489  18.365   2.512  1.00  0.00           H  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/home/exouser/miniconda3/envs/openff/lib/python3.12/site-packages/openmm/app/internal/pdbstructure

/media/volume/sdc/githubs/MinhSimPrep/bromodomains_testing/ligand_crys_H_SOL.pdb True


Generate a topology of the complex system

In [11]:
print(type(self.protein_solvated), type(self.ligand))

<class 'str'> <class 'openff.toolkit.topology.molecule.Molecule'>


In [12]:
complex_topology = self.generate_topologies(self.protein_solvated, ligand_to_insert=self.ligand)

Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule


In [14]:
interchange = self.top2interchange(complex_topology, ["openff-2.1.0.offxml", "opc-1.0.1.offxml", "ff14sb_off_impropers_0.0.3.offxml"])

In [ ]:
DEPRECATED_BELOW_THIS_LINE

In [15]:
class Simulation_Preparer(object):
    
    def __init__ (self, crystal_pdb_fn, ligand_resname):
        self.ligand_resname = ligand_resname
        
        # Split Crystal into protein and ligand pdbs based on selection strings
        #receptor_path, ligand_path = self.seperate_crys_using_MDA(crystal_pdb_fn, self.ligand_resname)
        
        # Convert ligand pdb above to an sdf with openbabel, addhs and make sdf
        #ligand_sdf_path = self.obabel_conversion((ligand_path,'ligand.sdf'))
        
        # The previous function also rewrites the input file with protons added at the following path
        #ligand_protonated = os.path.splitext(ligand_path)[0] + '_H' + os.path.splitext(ligand_path)[-1]
        
        #Verify existance of the protonated pdb and sdf
        #print(os.path.isfile(ligand_protonated), ligand_protonated, os.path.isfile(ligand_sdf_path), ligand_sdf_path)
        
        # Load an  openff Molecule from a SDF file
        self.ligand = Molecule.from_file(ligand_sdf_path)
        
        # Print out a SMILES code for the ligand
        print(self.ligand.to_smiles(explicit_hydrogens=False))
        
        # Protonate the receptor (ph7), then put it in a box of water
        #receptor_path = self.protonate_with_pdb2pqr(receptor_path)
        
        #self.receptor_solvated_fn = PDBFixer_solvation(receptor_path)
        # Create a phase of the ligand in a box of water
        #self.ligand_solvated_path = PDBFixer_solvation(ligand_protonated)

    def generate_topologies(self, save_as_jsons=True):
        #Create the topology of the complex phase
        comp_top = Topology.from_pdb(self.receptor_solvated_fn)
        #Insert the ligand into this phase and remove clashes
        self.comp_top = insert_molecule_and_remove_clashes(comp_top, self.ligand)
        # Do the same as above for the solvent phase, requires a slight workaround where the ligand is removed and added back in by openff
        solvent_phase_path = just_the_solvent(self.ligand_solvated_path, self.ligand_resname)
        solv_top = Topology.from_pdb(solvent_phase_path) #ligand cannot be here
        self.solv_top = insert_molecule_and_remove_clashes(solv_top, self.ligand) #and so is added here
        if save_as_jsons:
            with open("complex_topology.json", "w") as f:
                print(comp_top.to_json(), file=f)
            with open("solvent_topology.json", "w") as f:
                print(solv_top.to_json(), file=f)

    def generate_interchanges(self, xmls):
        sage_ff14sb = ForceField(*xmls)
        # Create interchanges of both phases (this takes a while)
        self.comp_interchange = sage_ff14sb.create_interchange(self.comp_top)
        self.solv_interchange = sage_ff14sb.create_interchange(self.solv_top)

    def openmm_writeout(self):
        make_simulation_and_writeout(self.comp_interchange, 'complex', self.ligand_resname)
        make_simulation_and_writeout(self.solv_interchange, 'solvent', self.ligand_resname)    
    
    def seperate_crys_using_MDA(crys_pdb_fn: str,
                                ligand_string: str,
                                receptor_string: str = 'protein',
                                ligand_pdb_fn: str = 'ligand.pdb',
                                receptor_pdb_fn: str = 'receptor.pdb'):
        """
        Loads the provided crystal structure, and seperates the atoms of resname LIGAND STRING into their own file
        Arguments:
            crys_pdb_fn: crystal structure to be seperated
            ligand_string: MDAnalysis selection string for ligand
            receptor_string: MDAnalysis selection string for receptor (default 'protein')
            ligand_pdb_fn: Write path for MDAnalysis ligand (default 'ligand.pdb')
            receptor_pdb_fn: Write path for MDAnalysis receptor (default 'receptor.pdb')
        Returns:
            receptor_pdb_fn: filename of the pdb file containing the receptor only
            ligand_pdb_fn: filename of the pdb file containing the atoms from the LIGAND_STRING selection
        """
        u = mda.Universe(crys_pdb_fn)
        all_atoms = u.select_atoms('all')
        receptor = all_atoms.select_atoms(receptor_string)
        ligand = all_atoms.select_atoms(ligand_string)
        receptor.write(receptor_pdb_fn)
        ligand.write(ligand_pdb_fn)
        return receptor_pdb_fn, ligand_pdb_fn
    
    
    def determine_restrained_residues(structure_file, n_closest, ligand_string):
        """
        Determines the 'n_closest' residues to the selection given by 'ligand_string' as measured by the distance of alpha
        carbons to the center of mass of the selection.  Assembles a selection string for the determined residues.
        EX.
        >>>determine_restrained_residues("SOME.pdb", 3, "resname lig")
        (resname PHE and resid 32) or (resname ALA and resid 47) or (resname GLY and resid 54)
        """
        u = mda.Universe(structure_file)
        protein_CAs = u.select_atoms('protein and name CA')
        uq = u.select_atoms(ligand_string)
        uq_com = uq.center_of_mass()
        #Build a list of resnames resids and distances (sort by distance)
        residues = []
        for atom in protein_CAs:
            dist = np.sqrt(np.sum((atom.position - uq_com)**2))
            residues.append([atom.resname, atom.resindex, dist])
        residues = sorted(residues, key = lambda x: x[2])
        #Craft the restraint string
        restraint_string = ''
        for res in residues[:n_closest]:
            restraint_string += f'(resname {res[0]} and resid {res[1]}) or '
        restraint_string = restraint_string[:-4]
        return restraint_string
    
    
    
    
    
    def protonate_with_pdb2pqr(protein_fn, protein_H_fn=None, at_pH=7):
        """Protonate the given structure using pdb2pqr30
        protein_fn: structure to be protonated
        protein_H_fn: filepath for protonated receptor (as pdb)
                        pqr output of pdb2pqr is inferred as protein_H_fn with a pqr extension instead of pdb
        at_pH: pH for protonation (default 7)
        """
        if protein_H_fn is None:
            protein_H_fn = os.path.splitext(protein_fn)[0] + '_H' + os.path.splitext(protein_fn)[-1]
        protein_pqr_fn = os.path.splitext(protein_H_fn)[0] + '.pqr'
        my_cmd = f'pdb2pqr30 --ff AMBER --nodebump --keep-chain --ffout AMBER --pdb-output {protein_H_fn} --with-ph {at_pH} {protein_fn} {protein_pqr_fn}'
        print('Protanting using command line')
        print(my_cmd)
        os.system(my_cmd)
        print('Done')
        return protein_H_fn
    
    
    def PDBFixer_solvation(in_pdbfile, solvated_file=None, padding=1.5, ionicStrength=0.15):
        """
        Generates a solvated system using PDBFixer.
        in_pdbfile: structure file: structure to be placed in a solvation box
        solvated_file: string: filename to save output.  Default is the add '_solvated' between the body and extension of the input file name
        padding: float or int: minimum nanometer distance between the boundary and any atoms in the input.  Default 1.5 nm = 15 A
        ionicStrength: float (not int as thats a lot of ions :) : molar strength of ionic solution. Default 0.15 M = 150 mmol
        """
        fixer = PDBFixer(in_pdbfile)
        fixer.addSolvent(padding=padding * openmm_unit.nanometer, ionicStrength=ionicStrength * openmm_unit.molar)
        # ADD PDBFixer hydrogens and parsing crystal structures (Hydrogens with pdb2pqr30 at the moment)
        if solvated_file == None:
            solvated_file = os.path.splitext(in_pdbfile)[0] + '_solvated' + os.path.splitext(in_pdbfile)[-1]
        with open(solvated_file, "w") as f:
            openmm.app.PDBFile.writeFile(fixer.topology, fixer.positions, f)
        return solvated_file
    
    
    def just_the_solvent(ligand_solvated_path, ligand_resname, solvent_phase_fn='solvent_phase.pdb'):
        u = mda.Universe(ligand_solvated_path)
        solvent = u.select_atoms(f'not {ligand_resname}')
        solvent.write(solvent_phase_fn)
        return solvent_phase_fn
    
    
    def insert_molecule_and_remove_clashes(topology: Topology, insert: Molecule, radius: Quantity = 1.5 * unit.angstrom,
                                           keep: list[Molecule] = [],) -> Topology:
        """
        Add a molecule to a copy of the topology, removing any clashing molecules.
    
        The molecule will be added to the end of the topology. A new topology is
        returned; the input topology will not be altered. All molecules that
        clash will be removed, and each removed molecule will be printed to stdout.
        Users are responsible for ensuring that no important molecules have been
        removed; the clash radius may be modified accordingly.
    
        Parameters
        ==========
        top
            The topology to insert a molecule into
        insert
            The molecule to insert
        radius
            Any atom within this distance of any atom in the insert is considered
            clashing.
        keep
            Keep copies of these molecules, even if they're clashing
        """
        # We'll collect the molecules for the output topology into a list
        new_top_mols = []
        # A molecule's positions in a topology are stored as its zeroth conformer
        insert_coordinates = insert.conformers[0][:, None, :]
        for molecule in topology.molecules:
            if any(keep_mol.is_isomorphic_with(molecule) for keep_mol in keep):
                new_top_mols.append(molecule)
                continue
            molecule_coordinates = molecule.conformers[0][None, :, :]
            diff_matrix = molecule_coordinates - insert_coordinates
    
            # np.linalg.norm doesn't work on Pint quantities 😢
            working_unit = unit.nanometer
            distance_matrix = (
                np.linalg.norm(diff_matrix.m_as(working_unit), axis=-1) * working_unit
            )
    
            if distance_matrix.min() > radius:
                # This molecule is not clashing, so add it to the topology
                new_top_mols.append(molecule)
            else:
                print(f"Removed {molecule.to_smiles()} molecule")
    
        # Insert the ligand at the end
        new_top_mols.append(insert)
    
        # This pattern of assembling a topology from a list of molecules
        # ends up being much more efficient than adding each molecule
        # to a new topology one at a time
        new_top = Topology.from_molecules(new_top_mols)
    
        # Don't forget the box vectors!
        new_top.box_vectors = topology.box_vectors
        return new_top
    
    
    def describe_state(state: openmm.State, name: str = "State"):
        max_force = max(np.sqrt(v.x**2 + v.y**2 + v.z**2) for v in state.getForces())
        print(f"{name} has energy {round(state.getPotentialEnergy()._value, 2)} kJ/mol ",
              f"with maximum force {round(max_force, 2)} kJ/(mol nm)")
    
    
    def construct_openmm_simulation_from_interchange(interchange):
        """Construct an openn simulation object from an openff interachange"""
        # Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
        integrator = openmm.LangevinIntegrator(300 * openmm_unit.kelvin, 1 / openmm_unit.picosecond, 0.001 * openmm_unit.picoseconds)
        # Under the hood, this creates *OpenMM* `System` and `Topology` objects, then combines them together
        simulation = interchange.to_openmm_simulation(integrator=integrator)
        # Add a reporter to record the structure every 10 steps
        dcd_reporter = openmm.app.DCDReporter(f"trajectory.dcd", 1000)
        simulation.reporters.append(dcd_reporter)
        simulation.reporters.append(openmm.app.StateDataReporter(sys.stdout, 1000, step=True, potentialEnergy=True, temperature=True))
        #Evaluate and Report pre-minimized energy
        describe_state(simulation.context.getState(getEnergy=True, getForces=True), "Original state")
        #Minimize the structure
        simulation.minimizeEnergy()
        #Evaluate and Report post-minimized energy
        describe_state(simulation.context.getState(getEnergy=True, getForces=True), "Minimized state")
        simulation.context.setVelocitiesToTemperature(300 * openmm_unit.kelvin)
        return simulation
    
    
    def make_simulation_and_writeout(interchange, phase, ligand_resname='resname UNK'):
        sim = make_openmm_simulation(interchange)
        with open(f'{phase}_final.xml', "w") as xml_file:
            xml_file.write(openmm.XmlSerializer.serialize(sim.system))
        
        with open(f'{phase}_final.pdb', "w") as pdb_file:  
            openmm.app.PDBFile.writeFile(sim.topology,
                                         sim.context.getState(getPositions=True, enforcePeriodicBox=True).getPositions(),
                                         file=pdb_file,
                                         keepIds=True)
        with open(f'{phase}_final.pdb', 'r') as f:
            lines = f.readlines()
        assert ligand_resname.startswith('resname') and len(ligand_resname) == 11
        ligand_resname = ligand_resname[-3:]
        
        lines = [line.replace('UNK', ligand_resname) for line in lines]
        with open(f'{phase}_final.pdb', 'w') as f:
            f.writelines(lines)
    
    
    
    def run_for_walltime(simulation, num_minutes):
        simulation.runForClockTime(num_minutes * openmm_unit.minute)
    
    
    def run_for_steps(simulation, nsteps):
        simulation.step(nsteps)


In [ ]:
prepper = Simulation_Preparer('3mxf.pdb', 'resname JQ1') # A Crystal Structure and A residue defining the ligand

In [ ]:
prepper.generate_topologies(save_as_jsons=True)

In [ ]:
prepper.generate_interchanges(["openff-2.1.0.offxml", "opc-1.0.1.offxml", "ff14sb_off_impropers_0.0.3.offxml"]) # Takes a while

In [ ]:
prepper.openmm_writeout() # Takes a while

In [ ]:
# ALL PREVIOUS MEMORY CAN BE CLEARED AT THIS POINT?

In [ ]:
complex_fns = ('complex_final.pdb', 'complex_final.xml')
solvent_fns = ('solvent_final.pdb', 'solvent_final.xml')
ligand_resname = 'resname UNK'
yank_output_dir = 'UNK/yankrun2'
yaml_file_fn = 'UNK/yank_script.yaml'
restraint_string = determine_restrained_residues(complex_fns[0], 4, ligand_resname)

with open(yaml_file_fn, 'w') as f:
    f.write(write_the_yaml(complex_fns, solvent_fns, ligand_resname, yank_output_dir, restraint_string))

In [ ]:
with open(yaml_file_fn, 'r') as f:
    my_yaml = yaml.load(f, Loader=YankLoader)
yaml_builder = yank.experiment.ExperimentBuilder(script=my_yaml)

In [ ]:
yaml_builder.run_experiments()

In [ ]:
import yank.analyze

In [ ]:
Analyzer = yank.analyze.ExperimentAnalyzer('UNK/yankrun2/experiments/')
results = Analyzer.auto_analyze()

In [ ]:
print(results)